In [3]:
#importing librarie
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

### Reading the dataframe

In [4]:
df=pd.read_excel("normal_shock.xlsx",engine='openpyxl')
#Normalizing the data between 0.1 to 0.9
result=normalized_df=(df-df.min())/(df.max()-df.min())*0.8+0.1
result

,M1,M2,p2/p1,ρ2/ρ1,T2/T1,p02/p01
0,0.100000,0.900000,0.100000,0.100000,0.100000,0.900000
1,0.102005,0.886641,0.100675,0.103354,0.101102,0.900000
2,0.104010,0.873700,0.101361,0.106728,0.102203,0.900000
3,0.106015,0.860898,0.102050,0.110122,0.103288,0.899915
4,0.108020,0.848513,0.102748,0.113516,0.104373,0.899915
...,...,...,...,...,...,...
395,0.891980,0.100835,0.886709,0.897289,0.887049,0.101705
396,0.893985,0.100696,0.890020,0.897972,0.890287,0.101279
397,0.895990,0.100417,0.893340,0.898654,0.893508,0.100853
398,0.897995,0.100278,0.896666,0.899337,0.896762,0.100426


### Taking Input from User

In [ ]:
array=result.to_numpy()
print(list(result.columns))
M=6                                 #Maxium output plus input=6
ip=int(input("No. of Inputs neurons = "))
op=M-ip
#op=int(input("No of Outputs neurons = "))
train=int(input("No of Training patterns out of total 400 = "))
test=int(input("No of Testing patterns = "))
h=int(input("No of Hidden neurons = "))
Target=[]
Input=[]
for row in array[:train]:
    Input.append(row[:ip])
    Target.append(row[ip:ip+op])
I=np.insert(Input,0,1,axis=1)
v=np.random.rand(ip+1,h)
w=np.random.rand(h+1,op)

### Defining Initial Variable

In [4]:
iteration=0
dw=np.zeros([h+1,op],dtype=float)
dv=np.zeros([ip+1,h],dtype=float)
alpha=0.5             #ASUUMING alpha =0.5
eta=0.25              #by optimization eta=0.25            
error_new=1


### Back Propogation Function

In [5]:
def back_propogation(ip,op,h,train,w,dw,v,dv):    #defining back propagation function
    for j in range(h+1):
        for k in range(op):                       #for input to hidden layers weightage updation
            Sum=0
            for p in range(train):
                Sum+=(Target[p][k]-O[p][k])*O[p][k]*(1-O[p][k])*OH[p][j]      #sum=differntiation of error wrt wjk
            w[j][k]=w[j][k]+eta*Sum/train+alpha*dw[j][k]
            dw[j][k]=eta*Sum/train
   
    for m in range(ip+1):                          #for hidden to output layer weightage updatation
        for j in range(h):
            sum=0
            for p in range(train):
                for k in range(op):                #sum=differntiation of error wrt vmj
     
                  sum+=(Target[p][k]-O[p][k])*O[p][k]*(1-O[p][k])*w[j][k]*OH[p][j]*(1-OH[p][j])*I[p][m]  
            v[m][j]=v[m][j]+sum*eta/(op*train)+alpha*dv[m][j]
            dv[m][j]=sum*eta/(op*train)
    return(w,dw,v,dv)
    
    
    
    

In [6]:
output_array=list()  #array to store incremental error value

### Training algorithm 

In [ ]:
while(iteration<20000 and error_new>.0000001):
    IH=np.dot(I,v)                                                          #input to hidden neuron
    O_H=[[1/(1+np.exp(-x)) for x in IH[i]]for i in range(len(IH))]          #log sigmoid function as TF
    OH=np.insert(O_H,0,1,axis=1)                                            #output of hidden neuron with bias
    IO=np.dot(OH,w)                                                         #input to output neuoron
    O=[[1/(1+np.exp(-x)) for x in IO[i]]for i in range(len(IO))]         # output of output neurons &log sigmoid function as TF
    error_new=mean_squared_error(Target,O)                                  
    w,dw,v,dv=back_propogation(ip,op,h,train,w,dw,v,dv)
    iteration=iteration+1
    output_array.append(error_new)
    print(error_new)
    
print('mean squared error = ',error_new)
print('no of iterations = ',iteration)
print('weight between input and hidden\n', v)
print('weight between hidden and output\n', w)

In [11]:
error=pd.DataFrame(output_array,columns=['MSE']).reset_index()  #chainging array to data frame for plotting

### Plot between error and iteration

In [ ]:
error.plot.line('index' ,'MSE')

### Testing the ANN

In [ ]:
#testing
T_test=[]
Input_test=[]
for row in array[train:train+test]:
    Input_test.append(row[:ip])
    T_test.append(row[ip:ip+op])
I_test=np.insert(Input_test,0,1,axis=1)
IH_test=np.dot(I_test,v)
O_H_test=[[1/(1+np.exp(-x)) for x in IH_test[i]]for i in range(len(IH_test))]
OH_test=np.insert(O_H_test,0,1,axis=1)
IO_test=np.dot(OH_test,w)
O_test=[[1/(1+np.exp(-x)) for x in IO_test[i]]for i in range(len(IO_test))]
error=mean_squared_error(T_test,O_test)
print('mean squared error of testing pattern = ',error)


### Converting values to De-Normalized form 

In [ ]:
#denormalize
max_value_array=[]
min_value_array=[]
for feature in df.columns[ip:ip+op]:
    max_value_array.append(df[feature].max())
    min_value_array.append(df[feature].min())
denormalized_output=[[(O_test[i][j]-0.1)/0.8*(max_value_array[j]-min_value_array[j])+min_value_array[j] for j in range(len(O_test[0]))]for i in range(len(O_test))]
pd.DataFrame(denormalized_output)